In [1]:
%reset
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np
import defmod as dm
import multimodule_usefulfunctions as mm 
import matplotlib.pyplot as plt
import defmod.hamiltonian_multishape as hamiltonian


torch.set_default_tensor_type(torch.DoubleTensor)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
nb_pts1 = 2
nb_pts2 = 3
nb_pts = [nb_pts1, nb_pts2]
dim = 2

o1 = torch.tensor([1.,1.])
o2 = torch.tensor([5.,5.])
origin = [o1,o2]
radius = [2.,1.]
numberPoints = [nb_pts1, nb_pts2]
gd_list = mm.multipleCircles(origin, radius, numberPoints)

In [3]:
# Defining the deformation modules
sigma1 = 2
manifold1 = dm.manifold.Landmarks(dim, nb_pts1, gd=gd_list[0].view(-1))
trans1 = dm.deformationmodules.Translations(manifold1, sigma1)

sigma2 = 2
manifold2 = dm.manifold.Landmarks(dim, nb_pts2, gd=gd_list[1].view(-1))
trans2 = dm.deformationmodules.Translations(manifold2, sigma2)

module_list = [trans1, trans2]
sigma_bg = 0.2
modules = dm.multishape.MultiShapeModule(module_list, sigma_bg)


In [4]:
#globaltrans
nb_pts1 = 10
nb_pts2 = 7
dim = 2

o1 = torch.tensor([0.,0.]).float()
o2 = torch.tensor([10.,0.]).float()
origin = [o1,o2]
r1 = 2
r2 = 1
radius = [r1, r2]
nb_pts = [nb_pts1, nb_pts2]
source = mm.multipleCircles(origin, radius, nb_pts)

origin_target = [[10., 0.], [0.,0.]]
target = mm.multipleCircles(origin_target, radius, nb_pts)

In [5]:
# global translation
sigma1 = 20
manifold1 = dm.manifold.Landmarks(dim, nb_pts1, gd=source[0].view(-1))
trans1 = dm.deformationmodules.GlobalTranslation(manifold1, sigma1)

sigma2 = 20
manifold2 = dm.manifold.Landmarks(dim, nb_pts2, gd=source[1].view(-1))
trans2 = dm.deformationmodules.GlobalTranslation(manifold2, sigma2)

module_list = [trans1, trans2]
sigma_bg = 1
modules = dm.multishape.MultiShapeModule(module_list, sigma_bg)

In [6]:
#globaltrans
constr_Id = dm.constraints.Identity()
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

gd0_list = [source[0].double(), source[1].double(), [source[0].double(), source[1].double()]]
mom0_list = [*[torch.rand(gd.shape) for gd in modules.manifold.gd[:-1]], [torch.rand(gd.shape) for gd in modules.manifold.gd[-1]]]

H.module.manifold.fill_gd([gd0_list[0].view(-1), gd0_list[1].view(-1), [gd0_list[2][0].view(-1), gd0_list[2][1].view(-1)]])
H.module.manifold.fill_cotan([mom0_list[0].view(-1), mom0_list[1].view(-1), [mom0_list[2][0].view(-1), mom0_list[2][1].view(-1)]])
H.geodesic_controls()

In [7]:
# Setting cotangent variables
p1 = torch.cat([torch.ones(nb_pts1,1, requires_grad=True), torch.zeros(nb_pts1,1)],1).view(-1)
p2 = torch.cat([-torch.ones(nb_pts2,1,requires_grad=True), torch.zeros(nb_pts2,1)],1).view(-1)
cotan = [p1, p2, [p1, p2]]

modules.manifold.fill_cotan(cotan)

In [8]:
constr_Id = dm.constraints.Identity()
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

In [9]:
# 2 Modules
def controlslist2tensor(controls_list):
    return torch.cat([controls_list[0].view(-1),controls_list[1].view(-1), controls_list[-1][0].view(-1),controls_list[-1][1].view(-1)])

def controlstensor2list(controls, nb_pts, dim):
    n = sum(nb_pts)
    return [controls[:dim*nb_pts[0]], controls[dim*nb_pts[0]:dim*(nb_pts[0]+nb_pts[1])], [controls[dim*n:dim*(n+nb_pts[0])], controls[dim*(n+nb_pts[0]):]]]

def gdtensor2list(gdtensor, nb_pts, dim):
    gdlist = []
    gdlist_bg = []
    j = 0
    n = sum(nb_pts)
    for i in range(len(nb_pts)):
        gdlist.append(gdtensor[dim*j:dim*(j+nb_pts[i])])
        gdlist_bg.append(gdtensor[dim*(n+j):dim*(n+j+nb_pts[i])])
        j = j + nb_pts[i]
    return [*gdlist, gdlist_bg]

def gdlist2tensor(gdlist):
    return torch.cat([*gdlist[:-1],*gdlist[-1]],0)

In [72]:
def test_gradcheck_list2tensor():
    def list2tensor(controls):
        
        controls = controlstensor2list(controls, [nb_pts1, nb_pts2], dim)
        
        return controlslist2tensor(controls)
    
    controls = torch.cat([torch.ones(modules.controls[0].shape, requires_grad=True), torch.ones(modules.controls[0].shape, requires_grad=True)])

    return torch.autograd.gradcheck(list2tensor, controls, raise_exception=True)

test_gradcheck_list2tensor()

True

In [73]:
def test_gradcheck_man_fill():
    def man_fill(gd):
        
        out = dm.manifold.Landmarks(2, 10)
        out.fill(man, copy=True)
        
        return out.gd
    
    return torch.autograd.gradcheck(man_fill, (gd), raise_exception=True)


man = dm.manifold.Landmarks(2, 10)
man.fill_gd(torch.rand(2*10, requires_grad=True))

gd = man.gd

#test_gradcheck_man_fill()

## return false is normal

In [74]:
def test_gradcheck_cot_to_vs():
    def cot_to_vs(cotan):  
        
        man = mod.manifold.copy()
        
        if j in [-1, 2]:
            cotan = controlstensor2list(cotan, nb_pts, dim)

        man.fill_cotan(cotan) 

        field = man.cot_to_vs(1)
        return field(points)
    
    return torch.autograd.gradcheck(cot_to_vs, (cotan), raise_exception=True)

modules.compute_geodesic_variables(constr_Id)

j = 2

mod = modules.module_list[j]
if j in [-1, 2]:
    cotan = controlslist2tensor(mod.manifold.cotan)
else:
    cotan = mod.manifold.cotan


points = torch.rand(dim*nb_pts1).view(-1,2)


test_gradcheck_cot_to_vs()

True

In [75]:
def test_gradcheck_innerprod():
    def innerprod(gd, mom):
        
        manifold.fill_gd(gd)
        manifold.fill_cotan(mom)  
                
        return mod.manifold.inner_prod_field(mod.field_generator())
    
    return torch.autograd.gradcheck(innerprod, (gd, mom), raise_exception=True)

manifold = dm.manifold.Landmarks(dim, nb_pts1)
mod = dm.deformationmodules.Translations(manifold, 1)

gd = torch.rand(nb_pts1*dim,1, requires_grad=True)
mom = torch.rand(nb_pts1*dim,1, requires_grad=True)

test_gradcheck_innerprod()

True

In [76]:
def test_gradcheck_autoaction():
    def autoaction(gd, mom):        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, nb_pts, dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, nb_pts, dim))
        
        return H.module.autoaction()

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)


    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

test_gradcheck_autoaction()

True

In [77]:
def test_gradcheck_geodesic_cont_self():
    def geodesic_cont_self(gd, mom):
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        H.module.compute_geodesic_control_from_self(H.module.manifold)

        return controlslist2tensor(H.module.controls)

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)

    return torch.autograd.gradcheck(geodesic_cont_self, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_cont_self()

True

In [78]:
def test_gradcheck_geodesic_controls():
    def geodesic_controls(gd, mom):        
        man = H.module.manifold.copy()
        H.module.manifold.fill_gd(gdtensor2list(gd, nb_pts, dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, nb_pts, dim))
        
        H.module.compute_geodesic_control(man)

        return controlslist2tensor(H.module.controls)

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)
    

    return torch.autograd.gradcheck(geodesic_controls, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_controls()

True

In [79]:
def test_gradcheck_geodesic_var():
    def geodesic_var(gd, mom):
        
        H.module.manifold.fill_gd(gdtensor2list(gd, nb_pts, dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, nb_pts, dim))

        H.geodesic_controls()
        return controlslist2tensor(H.module.controls)
        #return H.module.l

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)

    return torch.autograd.gradcheck(geodesic_var, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_var()

True

In [80]:
def test_gradcheck_apply_mom():
    def apply_mom(gd, mom):

        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))

        modules.compute_geodesic_variables( constr_Id)
        return H.apply_mom()

    
    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)


    return torch.autograd.gradcheck(apply_mom, (gd, mom), raise_exception=True)

test_gradcheck_apply_mom()

True

In [82]:
def test_gradcheck_Hamiltonian_2modules():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        H.geodesic_controls()
                
        return H()
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

#modules.compute_geodesic_variables(modules.manifold, constr_Id)

gd = gdlist2tensor(modules.manifold.gd)
mom = controlslist2tensor(modules.manifold.cotan)


test_gradcheck_Hamiltonian_2modules()

True

In [25]:
# check derivative of Hamiltonian wrt controls is zero

def Hamiltonian(controls):
    controls = controlstensor2list(controls, nb_pts, dim)
    modules.fill_controls(controls)
    H = hamiltonian.Hamiltonian_multi(modules, constr_Id)
    H.geodesic_controls()
    return H()

#controls = dm.usefulfunctions.flatten_tensor_list(modules.controls, [])
controls = controlslist2tensor(modules.controls)

#a = torch.autograd.grad(Hamiltonian(controls), controls, retain_graph=True)

#print(a)

In [26]:
def test_gradcheck_shooting():
    def shooting(gd, mom):
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        intermediate_states, intermediate_controls = dm.shooting.shoot_euler(H, it=2)
        return [H.module.manifold.cotan[0], H.module.manifold.cotan[1], H.module.manifold.cotan[2][0], H.module.manifold.cotan[2][1]]
    return torch.autograd.gradcheck(shooting, (gd, mom), raise_exception=True,atol=1e-2)

gd = gdlist2tensor(modules.manifold.gd)
mom = controlslist2tensor(modules.manifold.cotan)


test_gradcheck_shooting()

True

In [27]:
def test_gradcheck_copy():
    def copy(gd):
        man = dm.manifold.Landmarks(dim, nb_pts1, gd=gd_list[0].view(-1))
        man.fill_gd(gd)

        man_copy = man.copy(retain_grad=True)
        return man_copy.gd
    
    gd = manifold1.gd
    
    return torch.autograd.gradcheck(copy, gd)

test_gradcheck_copy()

True

In [10]:
def test_gradcheck_constraints():
    def constr(gd, controls):
        
        controls = controlstensor2list(controls, nb_pts, dim)
        modules.fill_controls(controls)
        
        gd = gdtensor2list(gd, nb_pts, dim)
        modules.manifold.fill_gd(gd)
        constr_Id = dm.constraints.Identity()
        return constr_Id(modules)
    
    gd = modules.manifold.gd
    controls = modules.controls
    
    gd_tensor = gdlist2tensor(gd)
    controls_tensor = controlslist2tensor(controls)
    
    return torch.autograd.gradcheck(constr, (gd_tensor, controls_tensor))
    
test_gradcheck_constraints()

True

In [11]:
constr_Id = dm.constraints.Identity()
my_model = dm.models.ModelMultishapePointsRegistration(gd_list, [trans1, trans2], sigma_bg,  gd_list, dm.attachement.L2NormAttachement_multi(), constr_Id)

def test_gradcheck_constraints():
    def constr(gd, controls):
        
        controls = controlstensor2list(controls, nb_pts, dim)
        modules.fill_controls(controls)
        
        gd = gdtensor2list(gd, nb_pts, dim)
        modules.manifold.fill_gd(gd)
        constr_Id = my_model.constraints
        return constr_Id(modules)
    
    gd = modules.manifold.gd
    controls = modules.controls
    
    gd_tensor = gdlist2tensor(gd)
    controls_tensor = controlslist2tensor(controls)
    
    return torch.autograd.gradcheck(constr, (gd_tensor, controls_tensor))
    
test_gradcheck_constraints()

TypeError: __init__() takes 6 positional arguments but 7 were given

In [20]:
print(gd_list, target
     )

[tensor([[ 3.0000,  1.0000],
        [-1.0000,  1.0000]], requires_grad=True), tensor([[6.0000, 5.0000],
        [4.5000, 5.8660],
        [4.5000, 4.1340]], requires_grad=True)] [tensor([[ 1.2000e+01,  0.0000e+00],
        [ 1.1618e+01,  1.1756e+00],
        [ 1.0618e+01,  1.9021e+00],
        [ 9.3820e+00,  1.9021e+00],
        [ 8.3820e+00,  1.1756e+00],
        [ 8.0000e+00,  2.4493e-16],
        [ 8.3820e+00, -1.1756e+00],
        [ 9.3820e+00, -1.9021e+00],
        [ 1.0618e+01, -1.9021e+00],
        [ 1.1618e+01, -1.1756e+00]], requires_grad=True), tensor([[ 1.0000,  0.0000],
        [ 0.6235,  0.7818],
        [-0.2225,  0.9749],
        [-0.9010,  0.4339],
        [-0.9010, -0.4339],
        [-0.2225, -0.9749],
        [ 0.6235, -0.7818]], requires_grad=True)]


In [ ]:
def test_gradcheck_model():
    def computemodel(gd):
        man1 = manifold1.copy()
        man2 = manifold2.copy()
        
        gd = gdtensor2list(gd, nb_pts, dim)
        man1.fill_gd(gd[0])
        man2.fill_gd(gd[1])
        
        mod1 = dm.deformationmodules.Translations(man1, sigma1)
        mod2 = dm.deformationmodules.Translations(man2, sigma2)
        
        constr_Id = dm.constraints.Identity()
        my_model = dm.models.ModelMultishapePointsRegistration(gd, [mod1, mod2], sigma_bg,  dm.attachement.L2NormAttachement_multi(), constr_Id)

        my_model.compute(target)
        
        #return [my_model.shot_manifold.gd[0], my_model.shot_manifold.gd[1], my_model.shot_manifold.gd[2][0], my_model.shot_manifold.gd[2][1]]
        return [my_model.shot_manifold.cotan[0], my_model.shot_manifold.cotan[1], my_model.shot_manifold.cotan[2][0], my_model.shot_manifold.cotan[2][1]]
        return [my_model.shot_manifold.cotan[0], my_model.shot_manifold.cotan[1], my_model.shot_manifold.cotan[2][0], my_model.shot_manifold.cotan[2][1]]

    
    gd = modules.manifold.gd
    controls = modules.controls
    
    gd_tensor = gdlist2tensor(gd)
    controls_tensor = controlslist2tensor(controls)
    
    return torch.autograd.gradcheck(computemodel, (gd_tensor))
    
test_gradcheck_model()

[tensor([[ 5.0589e+00,  8.4048e-18],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00,  2.3431e-16],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 4.6747e+00, -1.1952e+00]], grad_fn=<ViewBackward>), tensor([[ 7.1619e+00, -1.2557e-16],
        [ 6.7829e+00,  7.9172e-01],
        [ 5.9312e+00,  9.8725e-01],
        [ 5.2482e+00,  4.3937e-01],
        [ 5.2482e+00, -4.3937e-01],
        [ 5.9312e+00, -9.8725e-01],
        [ 6.7829e+00, -7.9172e-01]], grad_fn=<ViewBackward>), [tensor([[ 5.0589e+00,  8.4048e-18],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00,  2.3431e-16],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 

[tensor([[ 5.0589e+00,  1.0146e-06],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -2.6099e-09],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 4.6747e+00, -1.1952e+00]], grad_fn=<ViewBackward>), tensor([[ 7.1619e+00,  6.1270e-09],
        [ 6.7829e+00,  7.9172e-01],
        [ 5.9312e+00,  9.8725e-01],
        [ 5.2482e+00,  4.3937e-01],
        [ 5.2482e+00, -4.3937e-01],
        [ 5.9312e+00, -9.8725e-01],
        [ 6.7829e+00, -7.9172e-01]], grad_fn=<ViewBackward>), [tensor([[ 5.0589e+00,  1.0146e-06],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -2.6099e-09],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 

[tensor([[ 5.0589e+00,  8.2343e-11],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00,  1.2479e-09],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 4.6747e+00, -1.1952e+00]], grad_fn=<ViewBackward>), tensor([[ 7.1619e+00,  7.1695e-10],
        [ 6.7829e+00,  7.9172e-01],
        [ 5.9312e+00,  9.8725e-01],
        [ 5.2482e+00,  4.3937e-01],
        [ 5.2482e+00, -4.3937e-01],
        [ 5.9312e+00, -9.8725e-01],
        [ 6.7829e+00, -7.9172e-01]], grad_fn=<ViewBackward>), [tensor([[ 5.0589e+00,  8.2343e-11],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00,  1.2479e-09],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 

[tensor([[ 5.0589e+00,  1.3584e-09],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00,  2.1644e-10],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 4.6747e+00, -1.1952e+00]], grad_fn=<ViewBackward>), tensor([[ 7.1619e+00,  4.5834e-10],
        [ 6.7829e+00,  7.9172e-01],
        [ 5.9312e+00,  9.8725e-01],
        [ 5.2482e+00,  4.3937e-01],
        [ 5.2482e+00, -4.3937e-01],
        [ 5.9312e+00, -9.8725e-01],
        [ 6.7829e+00, -7.9172e-01]], grad_fn=<ViewBackward>), [tensor([[ 5.0589e+00,  1.3584e-09],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00,  2.1644e-10],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 

[tensor([[ 5.0589e+00,  2.6266e-09],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -8.7617e-10],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 4.6747e+00, -1.1952e+00]], grad_fn=<ViewBackward>), tensor([[ 7.1619e+00,  6.6202e-10],
        [ 6.7829e+00,  7.9172e-01],
        [ 5.9312e+00,  9.8725e-01],
        [ 5.2482e+00,  4.3937e-01],
        [ 5.2482e+00, -4.3937e-01],
        [ 5.9312e+00, -9.8725e-01],
        [ 6.7829e+00, -7.9172e-01]], grad_fn=<ViewBackward>), [tensor([[ 5.0589e+00,  2.6266e-09],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -8.7617e-10],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 

...

[tensor([[ 5.0589e+00,  1.6572e-10],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00,  3.0745e-09],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 4.6747e+00, -1.1952e+00]], grad_fn=<ViewBackward>), tensor([[ 7.1619e+00, -1.6609e-09],
        [ 6.7829e+00,  7.9172e-01],
        [ 5.9312e+00,  9.8725e-01],
        [ 5.2482e+00,  4.3937e-01],
        [ 5.2482e+00, -4.3937e-01],
        [ 5.9312e+00, -9.8725e-01],
        [ 6.7829e+00, -7.9172e-01]], grad_fn=<ViewBackward>), [tensor([[ 5.0589e+00,  1.6572e-10],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00,  3.0745e-09],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 

[tensor([[ 5.0589e+00, -5.3930e-11],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -3.9704e-11],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 4.6747e+00, -1.1952e+00]], grad_fn=<ViewBackward>), tensor([[ 7.1619e+00,  1.0125e-06],
        [ 6.7829e+00,  7.9172e-01],
        [ 5.9312e+00,  9.8725e-01],
        [ 5.2482e+00,  4.3937e-01],
        [ 5.2482e+00, -4.3937e-01],
        [ 5.9312e+00, -9.8725e-01],
        [ 6.7829e+00, -7.9172e-01]], grad_fn=<ViewBackward>), [tensor([[ 5.0589e+00, -5.3930e-11],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -3.9704e-11],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 

[tensor([[ 5.0589e+00, -1.1403e-09],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -1.0667e-09],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 4.6747e+00, -1.1952e+00]], grad_fn=<ViewBackward>), tensor([[ 7.1619e+00,  1.2339e-09],
        [ 6.7829e+00,  7.9172e-01],
        [ 5.9312e+00,  9.8725e-01],
        [ 5.2482e+00,  4.3937e-01],
        [ 5.2482e+00, -4.3937e-01],
        [ 5.9312e+00, -9.8725e-01],
        [ 6.7829e+00, -7.9172e-01]], grad_fn=<ViewBackward>), [tensor([[ 5.0589e+00, -1.1403e-09],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -1.0667e-09],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 

[tensor([[ 5.0589e+00, -1.8598e-08],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -1.8263e-08],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 4.6747e+00, -1.1952e+00]], grad_fn=<ViewBackward>), tensor([[ 7.1619e+00,  6.4909e-08],
        [ 6.7829e+00,  7.9172e-01],
        [ 5.9312e+00,  9.8725e-01],
        [ 5.2482e+00,  4.3937e-01],
        [ 5.2482e+00, -4.3937e-01],
        [ 5.9312e+00, -9.8725e-01],
        [ 6.7829e+00, -7.9172e-01]], grad_fn=<ViewBackward>), [tensor([[ 5.0589e+00, -1.8598e-08],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -1.8263e-08],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 

[tensor([[ 5.0589e+00, -6.6185e-09],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -6.4479e-09],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 4.6747e+00, -1.1952e+00]], grad_fn=<ViewBackward>), tensor([[ 7.1619e+00,  2.2124e-08],
        [ 6.7829e+00,  7.9172e-01],
        [ 5.9312e+00,  9.8725e-01],
        [ 5.2482e+00,  4.3937e-01],
        [ 5.2482e+00, -4.3937e-01],
        [ 5.9312e+00, -9.8725e-01],
        [ 6.7829e+00, -7.9172e-01]], grad_fn=<ViewBackward>), [tensor([[ 5.0589e+00, -6.6185e-09],
        [ 4.6747e+00,  1.1952e+00],
        [ 3.6690e+00,  1.9339e+00],
        [ 2.4257e+00,  1.9339e+00],
        [ 1.4199e+00,  1.1952e+00],
        [ 1.0357e+00, -6.4479e-09],
        [ 1.4199e+00, -1.1952e+00],
        [ 2.4257e+00, -1.9339e+00],
        [ 3.6690e+00, -1.9339e+00],
        [ 

In [51]:
def test_gradcheck_model():
    def test(gd):
        man1 = manifold1.copy()
        man2 = manifold2.copy()
        
        gd = gdtensor2list(gd, nb_pts, dim)
        man1.fill_gd(gd[0])
        man2.fill_gd(gd[1])
        
        mod1 = dm.deformationmodules.Translations(man1, sigma1)
        mod2 = dm.deformationmodules.Translations(man2, sigma2)
        
        constr_Id = dm.constraints.Identity()
        my_model = dm.models.ModelMultishapePointsRegistration(gd_list, [mod1, mod2], sigma_bg,  dm.attachement.L2NormAttachement_multi(), constr_Id)
        
        return my_model.test(gd_list)
    
    gd = modules.manifold.gd
    controls = modules.controls
    
    gd_tensor = gdlist2tensor(gd)
    controls_tensor = controlslist2tensor(controls)
    
    return torch.autograd.gradcheck(test, (gd_tensor))
    
test_gradcheck_model()

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[-7.3372e-02,  5.6862e-18,  7.3372e-02, -5.6862e-18],
        [ 0.0000e+00,  2.9486e-02,  0.0000e+00, -2.9486e-02],
        [ 7.3372e-02, -5.6862e-18, -7.3372e-02,  5.6862e-18],
        [ 0.0000e+00, -2.9486e-02,  0.0000e+00,  2.9486e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0

In [69]:
def test_gradcheck_model():
    def test(gd):
        man1 = manifold1.copy()
        man2 = manifold2.copy()
        
        gd = gdtensor2list(gd, nb_pts, dim)
        man1.fill_gd(gd[0])
        man2.fill_gd(gd[1])
        
        mod1 = dm.deformationmodules.Translations(man1, sigma1)
        mod2 = dm.deformationmodules.Translations(man2, sigma2)
        
        constr_Id = dm.constraints.Identity()
        my_model = dm.models.ModelCompoundWithPointsRegistration(gd_list, [mod1, mod2], [True, True],  dm.attachement.L2NormAttachement_multi())
        
        return my_model.test()
        
    
    gd = modules.manifold.gd
    controls = modules.controls
    
    gd_tensor = gdlist2tensor(gd)
    controls_tensor = controlslist2tensor(controls)
        
    return torch.autograd.gradcheck(test, (gd_tensor))
    
test_gradcheck_model()

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[-1.1243e-01,  2.3072e-02, -1.2537e-01, -2.2249e-02],
        [-2.1442e-01,  1.7113e-02,  6.2477e-02,  1.4991e-02],
        [ 1.9566e-01, -1.7572e-04, -1.4002e-02,  1.1506e-02],
        [ 7.5948e-03, -1.6956e-02, -3.6185e-02,  6.9628e-03],
        [ 4.0861e-02, -8.5072e-03,  7.2984e-03, -1.1258e-03],
        [ 1.2291e-01, -2.8066e-02, -1.4270e-03, -1.1491e-02],
        [-3.8702e-02, -8.3459e-03,  4.5139e-03, -7.6865e-03],
        [ 8.0470e-02, -7.3030e-02,  2.5101e-02, -1.4249e-02],
        [ 1.0525e-01, -2.6699e-03,  5.0804e-02,  1.3130e-02],
        [ 2.5409e-01,  1.5031e-01, -1.6091e-02,  1.3240e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0